# 08 Full Workflow (Experiment)
Read first: `examples/07_full_workflow.py`

Run the full prepare -> create_session -> execute path with a prompt you control.


## Select a provider
Set `PROVIDER_KEY` to `sonnet`, `opus`, `gpt`, or `gpt-codex` and run the next cell.
The helper will set env vars (prompting if missing) and print the selection.


In [ ]:
# Choose your provider key (sonnet | opus | gpt | gpt-codex)
PROVIDER_KEY = "sonnet"

In [ ]:
# Resolve foundation/provider paths and ensure env var
import importlib.util
import os
from pathlib import Path


def find_repo_root() -> Path:
    candidates = [Path.cwd(), Path.cwd().parent, Path.cwd().parent.parent]
    for p in candidates:
        if (p / "utils" / "providers.py").exists():
            return p
    return Path.cwd()


ROOT = find_repo_root()
providers_path = ROOT / "utils" / "providers.py"
spec = importlib.util.spec_from_file_location("providers_helper", providers_path)
module = importlib.util.module_from_spec(spec)
spec.loader.exec_module(module)
select_provider = module.select_provider
print_provider_menu = module.print_provider_menu

FOUNDATION_PATH, PROVIDER_PATH, REQUIRED_ENV = select_provider(PROVIDER_KEY, foundation="minimal")
print_provider_menu(PROVIDER_KEY)
print(f"Selected: {PROVIDER_KEY} -> {PROVIDER_PATH.name}")
env_set = "Yes" if os.environ.get(REQUIRED_ENV) else "No"
print(f"Required env: {REQUIRED_ENV} (set? {env_set})")

In [ ]:
# Optional: set API key here (safer: export in terminal)
import getpass
import os

if not os.getenv(REQUIRED_ENV):
    os.environ[REQUIRED_ENV] = getpass.getpass(f"Enter {REQUIRED_ENV}: ")
    print(f"Set {REQUIRED_ENV} for this kernel session.")
else:
    print(f"{REQUIRED_ENV} already set.")

In [ ]:
# Parameters (edit me)
PROMPT = "Summarize the benefits of composition over configuration in one paragraph."

In [ ]:
import time

from amplifier_foundation import load_bundle


async def run(prompt: str):
    start = time.time()
    foundation = await load_bundle(str(FOUNDATION_PATH))
    provider = await load_bundle(str(PROVIDER_PATH))
    composed = foundation.compose(provider)
    prepared = await composed.prepare()
    session = await prepared.create_session()
    async with session:
        response = await session.execute(prompt)
    elapsed = time.time() - start
    return response, elapsed


response, elapsed = await run(PROMPT)
print(f"Elapsed: {elapsed:.1f}s")
print("Response:")
print(response)

### Try this
- Swap `PROVIDER_PATH` to compare providers.
- Change `PROMPT` to include @mentions if you want to test mention loading (see the script for details).
- Wrap the run call in a loop to measure multiple prompts.
